In [32]:
import pandas as pd
import sys

In [35]:
# Go into above directory and download function
sys.path.append('../') 
import sheet_func

sheet_id = 867359064
# name = "par_algo"

df = sheet_func.sheet_to_df(sheet_id)
df

,,Family Name,Looked at?,Variation,Algo ID,Algorithm Description,Final Call,Exact Problem Statement?,Exact?,Time Complexity (Average),...,n = 10^6 value,n = 10^6 scale,n = 10^6 Rate,n = 10^9 value,n = 10^9 scale,n = 10^9 Rate,Starting Complexity,Remarks,Papers for ratio evaluations,Domains
0,1.0,Sorting,1.000,Comparison Sorting,10.0,NaN,NaN,1.0,1.0,O(n logn),...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
1,1.0,Sorting,1.000,Comparison Sorting,17.0,NaN,NaN,1.0,1.0,O(log² n),...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
2,1.0,Sorting,1.000,Comparison Sorting,19.0,NaN,NaN,1.0,1.0,O( n² ),...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
3,1.0,Sorting,0.001,Non-Comparison Sorting,25.0,NaN,NaN,1.0,1.0,O(n),...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Combinatorics
4,1.0,Sorting,0.001,Non-Comparison Sorting,NaN,integer sort,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NaN,All-Pairs Shortest Paths (APSP),0.000,NaN,NaN,Warshall-Floyd method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,All-Pairs Shortest Paths (APSP),0.000,NaN,NaN,Repeated plus-min method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,All-Pairs Shortest Paths (APSP),0.000,NaN,NaN,Warshall-Floyd method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,All-Pairs Shortest Paths (APSP),0.000,NaN,NaN,Repeated plus-min method,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
